In [1]:
import pandas as pd

# Load the Excel file
cols = ["보압시간", "사출속도1~4", "보압1~2"]
level_col_name = "변수 수준"
file_path = r"./data/사출 실험계획표 27.xlsx"


excel_data = pd.read_excel(file_path)[[level_col_name] + cols]
# # find the index of the first NaN value
null_idx = excel_data.index[excel_data.iloc[:, 0].isna()].tolist()[0]

# Display the first few rows of the dataframe to understand its structure
level_table = excel_data.iloc[:null_idx, :].copy()
data_table = (
    excel_data[cols].iloc[null_idx + 2 :, :].copy().reset_index(drop=True)
)
levels = level_table[level_col_name].tolist()

# Display the first few rows of the dataframe to understand its structure
assert data_table.shape[1] == len(
    cols
), "The number of columns is not correct"
print(f"Levels: {levels}")
print(f"Data table shape: {data_table.shape}")
level_table.head()

Levels: [-1, 0, 1]
Data table shape: (27, 3)


,변수 수준,보압시간,사출속도1~4,보압1~2
0,-1,1.2,40,10
1,0,1.4,50,20
2,1,1.6,60,30


In [2]:
# Function to map actual values to level values (-1, 0, 1)
def map_to_level(value, column):
    # Find the corresponding level for the value in the specified column
    level = level_table[level_table[column] == value][level_col_name].values[
        0
    ]
    return level


# Initialize an empty DataFrame with the same shape as data_table
mapped_data = pd.DataFrame(columns=cols, index=range(len(data_table)))

# Map each column in data_table to its corresponding level
for col in cols:
    mapped_data[col] = data_table[col].apply(lambda x: map_to_level(x, col))

# Convert the DataFrame to integer type
mapped_data = mapped_data.astype(int)

# Set the index name to "Case"
mapped_data.index.name = "Case"

# Set the indices to start from 1
mapped_data.index = mapped_data.index.map(lambda x: x + 1)

# Display the first few rows of the mapped data
assert mapped_data.shape[1] == len(
    cols
), "The number of columns is not correct"
mapped_data.head()

,보압시간,사출속도1~4,보압1~2
Case,,,
1,-1,-1,-1
2,0,-1,-1
3,1,-1,-1
4,-1,-1,0
5,0,-1,0


In [3]:
from pathlib import Path
from nn.schemas import _read_ss_curves, group_ss_curves

ss_curves = group_ss_curves(_read_ss_curves(raw_data_path=Path("data")))
ss_curves["Case"] = ss_curves.index.to_series().apply(
    lambda x: int(x.split("-")[1])
)
ss_curves = pd.merge(
    ss_curves.reset_index(drop=True),
    mapped_data,
    left_on="Case",
    right_index=True,
).drop(columns=["Case"])

ss_curves.head()

,strain,stress,보압시간,사출속도1~4,보압1~2
0,"[0.00, 0.00, 0.16, 0.20, 0.22, 0.26, 0.28, 0.3...","[0.000, 0.670, 0.760, 0.850, 0.940, 1.010, 1.1...",-1,-1,-1
1,"[0.00, 0.00, 8.00, 10.00, 11.00, 13.00, 14.00,...","[0.000, 0.359, 0.423, 0.487, 0.556, 0.605, 0.6...",-1,-1,-1
2,"[0.00, 0.00, 8.00, 10.00, 11.00, 13.00, 14.00,...","[0.000, 0.290, 0.354, 0.423, 0.497, 0.572, 0.6...",-1,-1,-1
3,"[0.00, 0.00, 8.00, 9.00, 11.00, 12.00, 14.00, ...","[0.000, 0.318, 0.385, 0.456, 0.513, 0.603, 0.6...",-1,-1,-1
4,"[0.00, 0.00, 8.00, 10.00, 11.00, 13.00, 14.00,...","[0.000, 0.369, 0.446, 0.526, 0.603, 0.664, 0.7...",-1,0,-1


In [4]:
import random
from typing import Tuple
import numpy as np
from nn.inference import inference


def pick_random_data(
    train_inputs: np.ndarray,
    train_outputs: np.ndarray,
    n: int = 1,
) -> Tuple[np.ndarray, np.ndarray]:
    x_test, y_test = train_inputs, train_outputs
    assert isinstance(x_test, np.ndarray) and isinstance(
        y_test, np.ndarray
    ), f"{type(x_test)} & {type(y_test)}"
    assert (
        x_test.shape[0] == y_test.shape[0]
    ), f"{x_test.shape} != {y_test.shape}"
    # pick n random data points
    idx = random.sample(range(x_test.shape[0]), n)
    return x_test[idx], y_test[idx]


def inference_lstm(
    test_data: Tuple[np.ndarray, np.ndarray],
    model_path: str,
    tolerance: float = 0.5,
    n: int = 5,
):
    x_test, y_test = test_data
    y_pred = inference(model_path=model_path, input_data=x_test)
    assert y_pred.shape == y_test.shape, f"{y_pred.shape} != {y_test.shape}"
    seq_len = y_pred.shape[1]

    def extract_points(y: np.ndarray):
        gap = seq_len // (n - 1)
        last_idx = seq_len - 1
        return tuple(y[0, min(i * gap, last_idx), 0] for i in range(n))

    y_pred_points = extract_points(y_pred)[1:]
    y_test_points = extract_points(y_test)[1:]
    print(f"prediction: {y_pred_points}, true: {y_test_points}")
    for yp, yt in zip(y_pred_points, y_test_points):
        assert abs(yp - yt) <= yt * tolerance, f"{yp} != {yt}"


def inference_ann(
    test_data: Tuple[np.ndarray, np.ndarray],
    model_path: str,
    tolerance: float = 0.5,
):
    x_test, y_test = test_data
    y_pred = inference(model_path, x_test)
    print(f"prediction: {y_pred}, true: {y_test}")

    # Check the predictions are within the tolerance
    strength_pred = float(y_pred[0][0])
    strength_true = float(y_test[0][0])
    assert (
        abs(strength_pred - strength_true) <= strength_true * tolerance
    ), f"{strength_pred} != {strength_true}"

    elongation_pred = float(y_pred[0][1])
    elongation_true = float(y_test[0][1])
    assert (
        abs(elongation_pred - elongation_true) <= elongation_true * tolerance
    ), f"{elongation_pred} != {elongation_true}"

In [5]:
import multiprocessing
from uuid import uuid4

from nn.ann import ANN
from nn.config import ANNModelConfig
from nn.dataloader import DataLoader
from nn.train import Trainer
from nn.utils.logger import ApiLogger

logger = ApiLogger(__name__)

epochs = 100
patience = 100
batch_size = 1
print_per_epoch = 1
ann_hyper_params = {
    "n1": [40],
    "n2": [30],
    "n3": [10],
}

# Creating a DataFrame
ann_df = ss_curves.copy()

# Calculating maximum of stress and strain for each row
ann_df["strength"] = ss_curves["stress"].apply(max)
ann_df["elongation"] = ss_curves["strain"].apply(max)

# Dropping the original strain and stress columns
ann_df.drop(["strain", "stress"], axis=1, inplace=True)

ann_x_data = ann_df[cols].astype(float).to_numpy()
ann_y_data = ann_df[["strength", "elongation"]].astype(float).to_numpy()

dim_out = ann_y_data.shape[1]
ann_model_config = ANNModelConfig(
    output_path=f".tmp/{uuid4().hex}",
    metrics=["mse", "mae", "mape"],
    kfold_splits=0,
    print_per_epoch=print_per_epoch,
    batch_size=batch_size,
    epochs=epochs,
    patience=patience,
    loss_funcs=["mape" for _ in range(dim_out)],
    loss_weights=[1 / dim_out for _ in range(dim_out)],
    l1_reg=None,
    l2_reg=None,
    dropout_rate=0.0,
    normalize_layer=False,
    dim_out=dim_out,
)
ann_data_loader = DataLoader(
    train_inputs=ann_x_data,
    train_outputs=ann_y_data,
    train_input_params=cols,
    train_output_params=["strength", "elongation"],
)
ann_trainer = Trainer(
    data_loader=ann_data_loader,
    model_class=ANN,
    model_name=ANN.__name__,
    model_config=ann_model_config,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=False,
)

ann_x_data.shape, ann_y_data.shape

((108, 3), (108, 2))

In [10]:
from functools import reduce
import json


num_hyper_params = reduce(
    lambda x, y: x * len(y), ann_hyper_params.values(), 1
)
ann_fstem = ""


for ann_fstem, phist in ann_trainer.hyper_train(ann_hyper_params):
    num_hyper_params -= 1

    json.dumps(phist["train_output"], indent=4)

    inference_ann(
        model_path=ann_fstem + ".keras",
        test_data=pick_random_data(ann_x_data, ann_y_data, n=1),
    )

assert ann_fstem != "", "fstem is empty"
assert num_hyper_params == 0, f"{num_hyper_params} != 0"
ann_fstem

[2023-11-23 22:15:29,494] nn.train:CRITICAL - model: ANN with 1 cases
[2023-11-23 22:15:29,495] nn.train:CRITICAL - training without multiprocessing...
[2023-11-23 22:15:29,559] nn.train:INFO - Loading model [.tmp\61556a412fb644f78ceba0d88e4ddab4\ANN_E100[N1=40][N2=30][N3=10]] with last: 100
[2023-11-23 22:15:29,561] nn.train:INFO - Already trained. Skipping...


1/1 [==============================] - 0s 32ms/step
prediction: [[10.244956 99.66468 ]], true: [[ 9.995 99.   ]]


'.tmp\\61556a412fb644f78ceba0d88e4ddab4\\ANN_E100[N1=40][N2=30][N3=10]'

In [11]:
from functools import reduce
import json
import multiprocessing
from uuid import uuid4
import numpy as np

import pandas as pd

from nn.config import LSTMModelConfig
from nn.dataloader import DataLoader
from nn.lstm import EmbeddingAttentionLSTMRegressor
from nn.schemas import normalize_1d_sequence
from nn.train import Trainer


epochs = 100
patience = 100
batch_size = 1
print_per_epoch = 1
seq_len = 64
lstm_hyper_params = {
    "seq_len": [seq_len],
}


lstm_x_data = ss_curves[cols].astype(float).to_numpy()
lstm_y_data = (
    ss_curves["stress"]
    .apply(lambda x: pd.Series(normalize_1d_sequence(x, seq_len)))
    .to_numpy()
)[:, :, np.newaxis]
# decoder_inputs = np.zeros_like(lstm_y_data)
# decoder_inputs[:, 1:, :] = lstm_y_data[:, :-1, :]  # Teacher forcing

assert lstm_x_data.shape[0] == lstm_y_data.shape[0], (
    f"Encoder input shape {lstm_x_data.shape} and decoder output shape {lstm_y_data.shape} "
    f"do not match"
)

lstm_model_config = LSTMModelConfig(
    output_path=f".tmp/{uuid4().hex}",
    metrics=["mse", "mae"],
    kfold_splits=0,
    print_per_epoch=print_per_epoch,
    batch_size=batch_size,
    epochs=epochs,
    patience=patience,
    loss_funcs=["mse"],
    loss_weights=[1.0],
    l1_reg=None,
    l2_reg=None,
    dropout_rate=0.0,
    normalize_layer=False,
    dim_out=1,
    ann_model_path=ann_fstem + ".keras",
)
lstm_data_loader = DataLoader(
    train_inputs=lstm_x_data,
    train_outputs=lstm_y_data,
    train_input_params=cols,
    train_output_params=["stress"],
)
lstm_trainer = Trainer(
    data_loader=lstm_data_loader,
    model_class=EmbeddingAttentionLSTMRegressor,
    model_name=EmbeddingAttentionLSTMRegressor.__name__,
    model_config=lstm_model_config,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=False,
)
lstm_x_data.shape, lstm_y_data.shape

((108, 3), (108, 64, 1))

In [7]:
num_hyper_params = reduce(
    lambda x, y: x * len(y), lstm_hyper_params.values(), 1
)

lstm_fstem = ""

for lstm_fstem, phist in lstm_trainer.hyper_train(lstm_hyper_params):
    num_hyper_params -= 1

    json.dumps(phist["train_output"], indent=4)

    inference_lstm(
        model_path=lstm_fstem + ".keras",
        test_data=pick_random_data(lstm_x_data, lstm_y_data, n=1),
    )


assert lstm_fstem != "", "fstem is empty"

assert num_hyper_params == 0, f"{num_hyper_params} != 0"
lstm_fstem

[2023-11-23 22:12:03,277] nn.train:CRITICAL - model: EmbeddingAttentionLSTMRegressor with 1 cases
[2023-11-23 22:12:03,278] nn.train:CRITICAL - training without multiprocessing...
[2023-11-23 22:12:03,316] nn.train:INFO - Start training: LSTMModelConfig(seed=777, print_per_epoch=1, output_path='.tmp/5dffab9e3202411da95ab5383ddd5240', metrics=['mse', 'mae'], epochs=100, batch_size=1, kfold_splits=0, patience=100, dim_in=10, dim_out=1, lr=0.001, loss_funcs=['mse'], loss_weights=[1.0], activation='relu', l1_reg=None, l2_reg=None, dropout_rate=0.0, normalize_layer=False, freeze_layers=[], seq_len=64, ann_model_path=None, state_transform_activation='tanh')
[2023-11-23 22:12:07,809] nn.callbacks:DEBUG - [Epoch   1  ]	rmse: 0.31006	loss: 0.09614	mse: 0.09614	mae: 0.19529	val_loss: 0.00645	val_mse: 0.00645	val_mae: 0.05052
[2023-11-23 22:12:10,232] nn.callbacks:DEBUG - [Epoch   2  ]	rmse: 0.08466	loss: 0.00717	mse: 0.00717	mae: 0.05184	val_loss: 0.00572	val_mse: 0.00572	val_mae: 0.04195
[2023-

KeyboardInterrupt: 